# From YouTube to Obsidian


Notebook to take a list of youtube links and convert them to markdown notes for Obsidian.


In [24]:
import json
import os
import re
import time
from datetime import UTC, datetime

import google.generativeai as genai
import yaml
from google.ai.generativelanguage_v1beta.types import content
from youtube_transcript_api import YouTubeTranscriptApi
from yt_dlp import YoutubeDL

In [2]:
# Load your Google API key
with open("config/credentials.yml", "r") as f:
    os.environ["GEMINI_API_KEY"] = yaml.safe_load(f)["GEMINI_API_KEY"]

Define system prompts


In [3]:
transcript_instruction = """
You will be given a transcript of a youtube video. 
Your task is make it natural, like a blog post. 
Add capitalizations, commas, stops or paragraph break when necessary. 
Only re-write the transcript, don't add anything else like "Yes, I can do that" or something like that.
And put a timestamp at the beginning of each paragraph so I can easily locate the paragraph in the video later.
"""

In [4]:
# Generate key takeaways and tags
available_tags = [
    "on/ai",
    "on/data-science",
    "on/programming",
    "on/software-engineering",
    "on/math",
    "on/learning",
    "on/knowledge",
    "on/productivity",
    "on/motivation",
    "on/goals",
    "on/health",
    "on/biohacking",
    "on/reviews",
    "on/relationships",
    "on/philosophy",
    "on/psychology",
    "on/communication",
    "on/creativity",
    "on/leadership",
    "on/management",
    "on/design",
    "on/history",
    "on/future",
    "on/pop-culture",
    "on/politics",
    "on/self-improvement",
]

available_tags_str = "\n".join(available_tags)

summary_instruction = f"""
You will be given a transcript of a youtube video along with the metadata. 
Your task is to write a TL;DR summary of the video (tweet length), the key takeaways, and the tags.
The summary should be short and to the point.
The key takeaways should be a list of bullet points.
The tags should be a list of tags (from one to three) from the following list: 
{available_tags_str}

You MUST use the "on/" prefix for the tags. 
For example, if the video is about AI, you should use "on/ai" as the tag.
"""

Create models and sessions


In [5]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

transcript_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    system_instruction=transcript_instruction,
)

In [6]:
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_schema": content.Schema(
        type=content.Type.OBJECT,
        enum=[],
        required=["summary", "key_takeaways", "tags"],
        properties={
            "summary": content.Schema(
                type=content.Type.STRING,
            ),
            "key_takeaways": content.Schema(
                type=content.Type.STRING,
            ),
            "tags": content.Schema(
                type=content.Type.ARRAY,
                items=content.Schema(
                    type=content.Type.STRING,
                ),
            ),
        },
    ),
    "response_mime_type": "application/json",
}

summary_model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    system_instruction=summary_instruction,
)

Load videos from videos.txt


In [20]:
def get_video_links() -> list:
    """Get video links from a txt file."""
    with open("input/play.txt", "r") as f:
        video_links = f.readlines()

    # Extract URLs from each line
    pat = re.compile(r"(https?://[^\s]+)")
    video_links = [pat.findall(line)[0] for line in video_links]
    # Remove duplicates while preserving order
    video_links = list(dict.fromkeys(video_links))
    return video_links


video_links = get_video_links()
print(video_links[:5])

['https://youtube.com/watch?v=N4YjXJVzoZY', 'https://youtube.com/watch?v=AOYUEqlWOGU', 'https://youtube.com/watch?v=3kKB6wYqP7Y', 'https://youtube.com/watch?v=vTmixSgeOI0', 'https://youtube.com/watch?v=YSMWN8VpY6A']


In [21]:
def fmt_time(seconds: int) -> str:
    """Convert seconds to MM:SS format."""
    return time.strftime("%M:%S", time.gmtime(seconds))


def get_note_title(video_title: str) -> str:
    """Get a valid note title from a video title."""
    # Remove special characters including dots and slashes, and emojis
    note_title = re.sub(r"[^\w\s']", "", video_title).capitalize()
    return note_title + ".md"

## Test with a single video


### 1. Create a dictionary with video metadata


In [25]:
opts = {"quiet": True, "noprogress": True}

print(f"Processing video: {video_links[0]}")
print("\tExtracting metadata...")
video_link = video_links[0]
with YoutubeDL(opts) as yt:
    info = yt.extract_info(video_link, download=False)
    views = info.get("view_count", 1)
    likes = info.get("like_count", 0)
    like_rate = round(100.0 * likes / views, 2)
    published = datetime.strptime(info.get("upload_date", ""), "%Y%m%d")
    video_data = {
        "id": info.get("id"),
        "title": info.get("title", ""),
        "author": info.get("channel", ""),
        "link": video_link,
        "likes": likes,
        "views": views,
        "like_rate": like_rate,
        "description": info.get("description", ""),
        "duration": round(info.get("duration", 0) / 60, 1),
        "published": published.strftime("%Y-%m-%d"),
        "created": datetime.now(tz=UTC).strftime("%Y-%m-%d"),
        "thumbnail": info.get("thumbnail", ""),
    }

print(json.dumps(video_data, indent=2))

Processing video: https://youtube.com/watch?v=N4YjXJVzoZY
	Extracting metadata...
{
  "id": "N4YjXJVzoZY",
  "title": "Mini Essays: The Ultimate Learning Tool",
  "author": "Odysseas",
  "link": "https://youtube.com/watch?v=N4YjXJVzoZY",
  "likes": 36067,
  "views": 457299,
  "like_rate": 7.89,
  "description": "About 8 months ago, I started using mini-essays as the core of my note-taking. I'll never turn back.\n\nI like to take my time with books, analyse their points and take time to write about them. It's high effort, sure, but the knowledge you get in return is priceless.\n\nMini-essays have been my best friend in this journey. They help me understand hard books, remember big ideas and form interesting connections, all with 30 minutes of time here and there.\n\nAnd that's only half the value. By writing mini-essays, I kill two birds with one stone. I'm on a journey to become a better writer, and these notes are a perfect way to practice writing and find my voice. Not bad, right?\n\

### 2. Extract the transcript


In [26]:
print("\tExtracting transcript...")
MAX_MINUTES_FOR_TRANSCRIPT = 15
try:
    transcript = YouTubeTranscriptApi.get_transcript(video_data["id"])
    transcript_text = "\n".join(
        [f"{fmt_time(entry['start'])} {entry['text']}" for entry in transcript]
    )
    video_data["transcript"] = transcript_text
except Exception as e:
    print(f"Error getting transcript: {e}")
    video_data["transcript"] = None
else:
    if video_data["duration"] < MAX_MINUTES_FOR_TRANSCRIPT:
        print("\tProcessing transcript...")
        chat_session = transcript_model.start_chat(history=[])
        new_transcript_text = chat_session.send_message(transcript_text).text
        video_data["new_transcript"] = new_transcript_text
    else:
        video_data["new_transcript"] = transcript_text

	Extracting transcript...


### 3. Generate summary and tags


In [27]:
print("\tGenerating summary and tags...")
try:
    video_data_text = (
        "Title: " + video_data["title"] + "\n"
        "Author: " + video_data["author"] + "\n"
        "Transcript:\n" + video_data["new_transcript"] + "\n"
    )
    chat_session = summary_model.start_chat(history=[])
    response = chat_session.send_message(video_data_text)
    video_data.update(json.loads(response.candidates[0].content.parts[0].text))
except Exception as e:
    print(f"Error generating summary: {e}")
    video_data["summary"] = video_data["description"]
    video_data["key_takeaways"] = ""
    video_data["tags"] = []

	Generating summary and tags...


### 4. Dump to markdown


In [28]:
print("\tDumping to markdown...")
note_title = get_note_title(video_data["title"])
note_path = os.path.join("output", note_title)
tags_str = ""
for tag in video_data["tags"]:
    tags_str += f'  - "{tag}"\n'

note = (
    "---\n"
    f'title: "{video_data["title"]}"\n'
    f'source: "{video_data["link"]}"\n'
    "author:\n"
    f'- "[[{video_data["author"]}]]"\n'
    f'published: {video_data["published"]}\n'
    f'created: {video_data["created"]}\n'
    f'description: "{video_data["summary"]}"\n'
    f'like_rate: {video_data["like_rate"]}\n'
    "tags:\n"
    f"{tags_str}\n"
    "---\n\n"
    f'[![Thumbnail]({video_data["thumbnail"]})]({video_data["link"]})\n\n'
    f'{video_data["summary"]}\n\n'
    "## Key takeaways\n"
    f'{video_data["key_takeaways"]}\n\n'
    "## Transcript\n"
    f'{video_data["new_transcript"]}'
)

with open("output/" + note_title, "w", encoding="utf-8") as f:
    f.write(note)
    print(f"\tNote saved to notes/{note_title}")

	Dumping to markdown...
	Note saved to notes/Mini essays the ultimate learning tool.md


# Main function and loop


In [ ]:
def get_video_data(video_link):
    opts = {"quiet": True, "noprogress": True}
    print("\tExtracting metadata...")
    with YoutubeDL(opts) as yt:
        info = yt.extract_info(video_link, download=False)
        video_id = info.get("id")
        video_data = {
            "id": video_id,
            "title": info.get("title", ""),
            "author": info.get("channel", ""),
            "link": video_link,
            "likes": info.get("like_count", 0),
            "views": info.get("view_count", 1),
            "like_rate": round(
                100.0 * info.get("like_count") / info.get("view_count"), 2
            ),
            "description": info.get("description", ""),
            "duration": info.get("duration", 0) / 60,
            "published": datetime.datetime.strptime(
                info.get("upload_date"), "%Y%m%d"
            ).strftime("%Y-%m-%d"),
            "created": datetime.datetime.now().strftime("%Y-%m-%d"),
            "thumbnail": info.get("thumbnail", ""),
        }

    return video_data


def enrich_video_data(video_data):
    print("\tExtracting transcript...")
    MAX_MINUTES_FOR_TRANSCRIPT = 15
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_data["id"])
        transcript_text = "\n".join(
            [f"{fmt_time(entry['start'])} {entry['text']}" for entry in transcript]
        )
        video_data["transcript"] = transcript_text
    except Exception as e:
        print(f"Error getting transcript: {e}")
        video_data["transcript"] = None
    else:
        if video_data["duration"] < MAX_MINUTES_FOR_TRANSCRIPT:
            print("\tProcessing transcript...")
            chat_session = transcript_model.start_chat(history=[])
            new_transcript_text = chat_session.send_message(transcript_text).text
            video_data["new_transcript"] = new_transcript_text
        else:
            video_data["new_transcript"] = transcript_text

    print("\tGenerating summary and tags...")
    try:
        video_data_text = (
            "Title: " + video_data["title"] + "\n"
            "Author: " + video_data["author"] + "\n"
            "Transcript:\n" + video_data["new_transcript"] + "\n"
        )
        chat_session = summary_model.start_chat(history=[])
        response = chat_session.send_message(video_data_text)
        video_data.update(json.loads(response.candidates[0].content.parts[0].text))
    except Exception as e:
        print(f"Error generating summary: {e}")
        video_data["summary"] = video_data["description"]
        video_data["key_takeaways"] = ""
        video_data["tags"] = []

    return video_data


def save_note(video_data):
    print("\tDumping to markdown...")
    note_title = video_data["title"].replace(".", "").replace("/", "") + ".md"
    tags_str = ""
    for tag in video_data["tags"]:
        tags_str += f'  - "{tag}"\n'

    note = (
        "---\n"
        f'title: "{video_data["title"]}"\n'
        f'source: "{video_data["link"]}"\n'
        "author:\n"
        f'- "[[{video_data["author"]}]]"\n'
        f'published: {video_data["published"]}\n'
        f'created: {video_data["created"]}\n'
        f'description: "{video_data["summary"]}"\n'
        f'like_rate: {video_data["like_rate"]}\n'
        "tags:\n"
        f"{tags_str}\n"
        "---\n\n"
        f'[![Thumbnail]({video_data["thumbnail"]})]({video_data["link"]})\n\n'
        f'{video_data["summary"]}\n\n'
        "## Key takeaways\n"
        f'{video_data["key_takeaways"]}\n\n'
        "## Transcript\n"
        f'{video_data["new_transcript"]}'
    )

    with open("notes/" + note_title, "w", encoding="utf-8") as f:
        f.write(note)
        print(f"\tNote saved to notes/{note_title}")

In [ ]:
with open("videos.txt", "r") as f:
    video_links = f.readlines()

video_links = [link.strip() for link in video_links]
print("No videos to process: ", len(video_links))

In [ ]:
for i, video_link in enumerate(video_links, 1):
    print(f"Processing video {i}/{len(video_links)}: {video_link}")
    video_data = get_video_data(video_link)

    # Check if already exists
    note_title = video_data["title"].replace(".", "").replace("/", "") + ".md"
    if os.path.exists("notes/" + note_title):
        print("\tNote already exists. Skipping...")
        continue

    video_data = enrich_video_data(video_data)
    save_note(video_data)
    print("\n")